# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# read in csv
filepath = 'weather_data.csv'

df = pd.read_csv(filepath)
df

,Unnamed: 0,City,Country,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness
0,0,Punta Arenas,CL,-53.1500,-70.9167,1653147446,4.61,997,87,39.31,0
1,1,Yeppoon,AU,-23.1333,150.7333,1653147447,19.37,1016,90,65.98,100
2,2,Upernavik,GL,72.7868,-56.1549,1653147449,11.56,1009,81,27.34,100
3,3,Necochea,AR,-38.5473,-58.7368,1653147450,11.34,1018,57,59.77,0
4,4,Vao,NC,-22.6667,167.4833,1653147451,28.07,1012,91,72.66,100
...,...,...,...,...,...,...,...,...,...,...,...
548,548,Cuamba,MZ,-14.8031,36.5372,1653148239,2.98,1015,59,71.71,4
549,549,Bosaso,SO,11.2842,49.1816,1653148241,0.49,1006,45,88.32,86
550,550,Berdigestyakh,RU,62.0989,126.6992,1653148242,3.06,1009,84,38.93,11
551,551,Tineo,ES,43.3376,-6.4145,1653148243,3.96,1016,71,55.65,100


In [3]:
df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness
0,0,Punta Arenas,CL,-53.1500,-70.9167,1653147446,4.61,997,87,39.31,0
1,1,Yeppoon,AU,-23.1333,150.7333,1653147447,19.37,1016,90,65.98,100
2,2,Upernavik,GL,72.7868,-56.1549,1653147449,11.56,1009,81,27.34,100
3,3,Necochea,AR,-38.5473,-58.7368,1653147450,11.34,1018,57,59.77,0
4,4,Vao,NC,-22.6667,167.4833,1653147451,28.07,1012,91,72.66,100


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553 entries, 0 to 552
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   553 non-null    int64  
 1   City         553 non-null    object 
 2   Country      552 non-null    object 
 3   Latitude     553 non-null    float64
 4   Longitude    553 non-null    float64
 5   Date         553 non-null    int64  
 6   Wind_Speed   553 non-null    float64
 7   Pressure     553 non-null    int64  
 8   Humidity     553 non-null    int64  
 9   Temperature  553 non-null    float64
 10  Cloudiness   553 non-null    int64  
dtypes: float64(4), int64(5), object(2)
memory usage: 47.6+ KB


In [5]:
# drop NA values
df2 = df.dropna().reset_index(drop = True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552 entries, 0 to 551
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   552 non-null    int64  
 1   City         552 non-null    object 
 2   Country      552 non-null    object 
 3   Latitude     552 non-null    float64
 4   Longitude    552 non-null    float64
 5   Date         552 non-null    int64  
 6   Wind_Speed   552 non-null    float64
 7   Pressure     552 non-null    int64  
 8   Humidity     552 non-null    int64  
 9   Temperature  552 non-null    float64
 10  Cloudiness   552 non-null    int64  
dtypes: float64(4), int64(5), object(2)
memory usage: 47.6+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# From Week 6 Day 3 Activity 7 Prof Booth
# Configure gmaps

gmaps.configure(api_key=g_key)

In [7]:
locations = df2.loc[:, ["Latitude", "Longitude"]]

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=df2['Humidity'],
    max_intensity=125, point_radius=20
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Create Masks for the following critera: temp between 70 and 80, wind less than 10 mph, 0 cloudiness

mask = (df2.Temperature < 80) & (df2.Temperature > 70) & (df2.Wind_Speed < 10) & (df.Cloudiness == 0)
df3 = df2.loc[mask]
df3.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness
9,9,Praxédis Guerrero,MX,31.3667,-106.0167,1653147459,6.76,1012,17,74.66,0
12,12,Cabo San Lucas,MX,22.8909,-109.9124,1653147281,9.22,1009,68,72.23,0
58,58,Alamogordo,US,32.8995,-105.9603,1653147531,9.22,1016,32,73.15,0
75,75,Bethel,US,41.3712,-73.4140,1653147553,8.05,1018,74,76.32,0
110,110,Saint-Pierre,RE,-21.3393,55.4781,1653147601,6.91,1018,73,74.16,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotels = []
ratings = []
addresses = []
city = []
country = []
lat = []
lng = []

# Loop through dataframe to grab lat and long for locations and google api request
# Prof Booth
for _, row in df3.iterrows():
    print(row.City)

    # grab lat and long and storing as variable
    target_coordinates = f"{row.Latitude}, {row.Longitude}"
    target_search = ""

    # set vaariables and params for API request
    target_radius = 5000
    target_type = "lodging"
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # API request
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(url, params = params)

    data = response.json()

    hotel = data["results"][0]["name"]
    address = data["results"][0]["vicinity"]
    rating = data["results"][0]["rating"]
    
    # put it all in a list
    hotels.append(hotel)
    ratings.append(rating)
    addresses.append(address)
    city.append(row.City)
    country.append(row.Country)
    lat.append(row.Latitude)
    lng.append(row.Longitude)

Praxédis Guerrero
Cabo San Lucas
Alamogordo
Bethel
Saint-Pierre
Mahon
Nhulunbuy
Ambon City
Carballo
Nīlēshwar
Langsa
Farafangana
São Domingos do Maranhão
Sisophon
Concarneau
Cuamba


In [10]:
# Create the hotel_df DataFrame

hotel_df = pd.DataFrame()
hotel_df['Hotel'] = hotels
hotel_df['Address'] = addresses
hotel_df['Rating'] = ratings
hotel_df['City'] = city
hotel_df['Country'] = country
hotel_df['Latitude'] = lat
hotel_df['Longitude'] = lng

hotel_df

,Hotel,Address,Rating,City,Country,Latitude,Longitude
0,Congregación placitas de Otero,Praxedis G. Guerrero,5.0,Praxédis Guerrero,MX,31.3667,-106.0167
1,Hotel Tesoro Los Cabos,"Boulevard Paseo de la Marina Lote 9 y 10, Cent...",4.4,Cabo San Lucas,MX,22.8909,-109.9124
2,Fairfield Inn & Suites by Marriott Alamogordo,"300 Panorama Boulevard, Alamogordo",4.0,Alamogordo,US,32.8995,-105.9603
3,Courtyard by Marriott Danbury,"3 Eagle Road, Danbury",4.3,Bethel,US,41.3712,-73.4140
4,Alize Plage,"17 Bis Boulevard Hubert Delisle, Saint-Pierre",3.9,Saint-Pierre,RE,-21.3393,55.4781
5,ARTIEM Capri,"Carrer de Sant Esteve, 8, Maó",4.4,Mahon,ES,39.8885,4.2658
6,East Arnhem Real Estate,"6 Traeger Close, Nhulunbuy",4.1,Nhulunbuy,AU,-12.2333,136.7667
7,Swiss-Belhotel Ambon,"Jalan Benteng Kapaha No.88, Uritetu",4.3,Ambon City,ID,-3.6954,128.1814
8,Pazo do Souto,Torre,4.6,Carballo,ES,43.2130,-8.6910
9,Neeleshwar Hermitage,"Ozhinhavalappu, Kasaragod District, Near, Nile...",4.6,Nīlēshwar,IN,12.2500,75.1000


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [19]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=df3['Humidity'],
    max_intensity=125, point_radius=20
)

fig = gmaps.figure()

# add layers
fig.add_layer(marker_layer)
fig.add_layer(heatmap_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))